## Le WebScraping en étapes:

Pour les bibliothèques nécessaires pour les exemples ci-dessous, vous pouvez accéder à votre ligne de commande et les installer avec Conda install (si vous utilisez la distribution Anaconda) ou pip install pour les autres distributions Python.

1.) **Requests** : Ce module est utilisé pour visiter une URL et obtenir le contenu d'une page web.La bibliothèque "Requests" en Python est un outil populaire utilisé pour simplifier les opérations liées aux requêtes HTTP. Elle offre une interface simple et élégante pour envoyer des requêtes HTTP, gérer les cookies, les en-têtes, les sessions, et gérer d'autres aspects liés aux communications web. "Requests" permet aux développeurs d'interagir facilement avec des API, de récupérer des données à partir de sites web, et d'effectuer diverses opérations liées aux protocoles HTTP. Grâce à sa simplicité d'utilisation et à ses fonctionnalités puissantes, "Requests" est largement utilisée dans le domaine du web scraping, de l'automatisation web, et de l'accès aux services web.

Vous pouvez le télécharger avec pip install requests ou conda install requests (pour la distribution Anaconda de Python) dans votre invite de commande.

2.) **BeautifulSoup** :une bibliothèque Python qui facilite l'extraction d'informations à partir de documents HTML et XML. Elle offre des méthodes souples pour parcourir et rechercher des éléments dans le code source HTML/XML, facilitant ainsi l'analyse et la manipulation de données web..

Vous pouvez le télécharger avec pip install beautifulsoup4 ou conda install beautifulsoup4 (pour la distribution Anaconda de Python) dans votre invite de commande.

3.) **parser HTML** :Beautiful Soup prend en charge différents parseurs (analysateurs) pour analyser le code source HTML ou XML. Deux des parseurs populaires utilisés avec Beautiful Soup sont "lxml" et "html.parser"...

* lxml : "lxml" est un parseur externe qui est construit sur les bibliothèques C libxml2 et libxslt. Il est rapide, efficace et capable de gérer des documents HTML et XML complexes. "lxml" est souvent recommandé pour sa rapidité, en particulier lors du traitement de grandes quantités de données.


* html.parser : "html.parser" est un parseur natif inclus dans la bibliothèque standard de Python. Il est plus lent que "lxml" mais peut être suffisant pour des tâches moins gourmandes en ressources. Aucune installation supplémentaire n'est nécessaire, car il fait partie de la bibliothèque standard.


* html5lib : Il s'agit d'un parseur externe qui implémente l'algorithme de l'HTML Living Standard. Il est précis et capable de gérer des documents HTML mal formés, mais il est généralement plus lent que les autres parseurs.

## Application:

On considère le site: www.imdb.com

**1- Analyse du Site Web :**

On considère le site imdb et cette URL : https://www.imdb.com/title/tt0108778/episodes/ .inspectez la structure HTML pour comprendre comment les informations de la série ,ses épisodes et les différentes saisons sont présentées.
Identifiez les balises HTML, les classes et les IDs pertinents qui contiennent les données que vous souhaitez extraire.

**2-Extraction des Informations de Base :**

* 2.1- les épisodes de la saison 1:

Utilisez Beautiful Soup pour envoyer une requête HTTP à la page https://www.imdb.com/title/tt0108778/episodes/  récupérez le contenu HTML.
Récupérez la liste des episodes de la 1ere saison.
Récupérer pour chaque épisode , son numéro, son nom, la date de sortie, la note et la description

* 2.2-Récupérer toutes les saisons  

Etendre votre travail précédent pour extraire les les épisodes des 10 saisons de la série



In [76]:
import requests
from bs4 import BeautifulSoup
from typing import Optional
import pandas as pd

In [18]:
# L'url du site à scrapper
url = 'https://www.imdb.com/title/tt0108778/episodes/'

# ma requete HTTP avec un "GET" au serveur du site identifier dans l'url
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3,content-Type=application/json'}
response = requests.get(url, headers=headers)

# J'affiche l'url ainsi que le retour du serveur
print(url, response.status_code)



https://www.imdb.com/title/tt0108778/episodes/ 200


In [3]:
# Je demande à beautifulSoup de conserver dans une variable "soup" la page web à scrapper (un script html)
soup = BeautifulSoup(response.content, 'html')
# soup

In [4]:
soup.select_one('h2').text

'Friends'

In [125]:
titres = soup.find_all('div', {'class': 'sc-ccd6e31b-4 eMYVLm'})
# titres.find_all('div', {'class': 'sc-ccd6e31b-5 cXcoHr'})
titres[0].find('div', {'class': 'ipc-title__text'})
# titres[0].find('span', {'class': 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'}).text

<div class="ipc-title__text">S1.E1 ∙ The One Where Monica Gets a Roommate</div>

In [147]:
def extract_info(url: str, header:Optional[dict]=None) -> list[str]:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    results = {}
    if response.status_code != 200:
        print(response.status_code)
        exit(1)
    soup = BeautifulSoup(response.content, 'html')
    episodes = soup.find_all('div', {'class': 'sc-ccd6e31b-4 eMYVLm'})
    for episode in episodes:
        try:
            infos = episode.find('div', {'class': 'ipc-title__text'}).text
            number,title=infos.split('∙')
            date= episode.find('span', {'class': 'sc-ccd6e31b-10 fVspdm'}).text
            note=episode.find('span', {'class': 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'}).text
            results[number]={'title':title.strip(), 'date':date.strip(), 'rating':note.strip()}
        except Exception as e:
            pass
    return results

In [148]:
url='https://www.imdb.com/title/tt0108778/episodes/?season=1'
extract_info(url)

{'S1.E1 ': {'title': 'The One Where Monica Gets a Roommate',
  'date': 'Mon, Apr 15, 1996',
  'rating': '8.1/10\xa0(10K)'},
 'S1.E2 ': {'title': 'The One with the Sonogram at the End',
  'date': 'Wed, Sep 28, 1994',
  'rating': '7.9/10\xa0(8K)'},
 'S1.E3 ': {'title': 'The One with the Thumb',
  'date': 'Wed, Oct 5, 1994',
  'rating': '8.0/10\xa0(7.5K)'},
 'S1.E4 ': {'title': 'The One with George Stephanopoulos',
  'date': 'Wed, Oct 12, 1994',
  'rating': '7.9/10\xa0(7.3K)'},
 'S1.E5 ': {'title': 'The One with the East German Laundry Detergent',
  'date': 'Wed, Oct 19, 1994',
  'rating': '8.3/10\xa0(7.3K)'},
 'S1.E6 ': {'title': 'The One with the Butt',
  'date': 'Wed, Oct 26, 1994',
  'rating': '8.0/10\xa0(6.9K)'},
 'S1.E7 ': {'title': 'The One with the Blackout',
  'date': 'Wed, Nov 2, 1994',
  'rating': '8.9/10\xa0(8.7K)'},
 'S1.E8 ': {'title': 'The One Where Nana Dies Twice',
  'date': 'Wed, Nov 9, 1994',
  'rating': '7.9/10\xa0(6.8K)'},
 'S1.E9 ': {'title': 'The One Where Underdog 

In [165]:
seasons = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

for id_season in range(1, 11):
    url = f'https://www.imdb.com/title/tt0108778/episodes/?season={id_season}'
    # saisons.append(extract_info(url,headers))
    seasons.update(extract_info(url, headers))

In [170]:
pd.DataFrame.from_dict(seasons,orient='index')

,title,date,rating
S1.E1,The One Where Monica Gets a Roommate,"Mon, Apr 15, 1996",8.1/10 (10K)
S1.E2,The One with the Sonogram at the End,"Wed, Sep 28, 1994",7.9/10 (8K)
S1.E3,The One with the Thumb,"Wed, Oct 5, 1994",8.0/10 (7.5K)
S1.E4,The One with George Stephanopoulos,"Wed, Oct 12, 1994",7.9/10 (7.3K)
S1.E5,The One with the East German Laundry Detergent,"Wed, Oct 19, 1994",8.3/10 (7.3K)
...,...,...,...
S10.E13,The One Where Joey Speaks French,"Wed, Feb 18, 2004",8.3/10 (4.9K)
S10.E14,The One with Princess Consuela,"Wed, Feb 25, 2004",8.4/10 (4.8K)
S10.E15,The One Where Estelle Dies,"Wed, Apr 21, 2004",8.4/10 (4.5K)
S10.E16,The One with Rachel's Going Away Party,"Wed, Apr 28, 2004",8.8/10 (5.2K)


In [163]:
#depleted
# rows = []
# for season in seasons:
#     for episode, details in season.items():
#         row = [episode.strip(), details['title'], details['date'], details['rating']]
#         rows.append(row)

# Define the columns
# columns = ['episode', 'title', 'date', 'rating']
# 
# # Create the DataFrame
# df = pd.DataFrame(rows, columns=columns)
# df

,episode,title,date,rating
0,S1.E1,The One Where Monica Gets a Roommate,"Mon, Apr 15, 1996",8.1/10 (10K)
1,S1.E2,The One with the Sonogram at the End,"Wed, Sep 28, 1994",7.9/10 (8K)
2,S1.E3,The One with the Thumb,"Wed, Oct 5, 1994",8.0/10 (7.5K)
3,S1.E4,The One with George Stephanopoulos,"Wed, Oct 12, 1994",7.9/10 (7.3K)
4,S1.E5,The One with the East German Laundry Detergent,"Wed, Oct 19, 1994",8.3/10 (7.3K)
...,...,...,...,...
229,S10.E13,The One Where Joey Speaks French,"Wed, Feb 18, 2004",8.3/10 (4.9K)
230,S10.E14,The One with Princess Consuela,"Wed, Feb 25, 2004",8.4/10 (4.8K)
231,S10.E15,The One Where Estelle Dies,"Wed, Apr 21, 2004",8.4/10 (4.5K)
232,S10.E16,The One with Rachel's Going Away Party,"Wed, Apr 28, 2004",8.8/10 (5.2K)


In [164]:
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(service=service, options=options)
wd.get("https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com")
wd.title

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


## Ressources utiles



https://www.crummy.com/software/BeautifulSoup/bs4/doc/

https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors


